# Controlling external tools

## Intro to Python's `subprocess` module

Python's `subprocess` module gives you access to calling external tools on your system. In the simplest case this means calling a command on your system and checking the output. This can be done using the `call` function.

When calling a system function with `subprocess` the process and any arguments are passed as a list of strings. This avoids the need for having to quote strings with spaces and special characters.

In [2]:
import subprocess

# Windows:
subprocess.call(['ipconfig', '/all'])
# Linux / macOS: use this instead:
# subprocess.call(['ifconfig', '-a'])

0

The response from the `call` method is the system exit code. If the process completed successfully this will be `0`. Any other value indicates an error, e.g.:

In [4]:
subprocess.call(['exit', '1'], shell=True)

1

Passing the `shell=True` argument executes the command inside a shell instance, giving it access to shell environment variables and commands. This can be a security hazard, opening your system up to Shell Injection. As a general rule, do not allow your code to call a subprocess with `shell=True` where the command parameters are not set or trusted by you.

At this point you still haven't seen the output of this command. If the output of the command is important, rather than using `call`, the `check_output` command will be more useful:

In [5]:
# Windows:
print(subprocess.check_output(['ipconfig']))
# Linux / macOS:
# print(subprocess.check_output(['ifconfig']))


Windows IP Configuration


Ethernet adapter Ethernet:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::c42a:e633:dbe5:1438%2
   IPv4 Address. . . . . . . . . . . : 10.0.2.15
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 10.0.2.2

Tunnel adapter Teredo Tunneling Pseudo-Interface:

   Connection-specific DNS Suffix  . : 
   IPv6 Address. . . . . . . . . . . : 2001:0:9d38:6abd:1099:1929:f5ff:fdf0
   Link-local IPv6 Address . . . . . : fe80::1099:1929:f5ff:fdf0%4
   Default Gateway . . . . . . . . . : ::



Unlike `call`, `check_output` will raise an Exception when there is an error. As such it is often the preferred method of using `subprocess`.

In [6]:
subprocess.check_output(['exit', '1'], shell=True)

CalledProcessError: Command '['exit', '1']' returned non-zero exit status 1

### Exercise: Displaying the current system processes

The MS DOS `tasklist` command lists the current processes on your machine. Use `check_output` to print a list of active processes on your machine.

Extension: Add the `/?` argument to `tasklist` to see the help. Read the help
and filter the tasks for `python.exe` processes.

In [7]:
#%load 4000_solution_01.py

## Interacting with open processes

You've seen the simple pattern for firing a process and waiting for a response. What about the more complex pattern of communicating with a process? For this we use pipes and the `Popen` class. If you haven't heard of a pipe in reference to computer programming before, a pipe is a one way communication channel that may be used for interprocess communication. For our purposes we assume that programs we want to communicate with have three pipes:

1. The `stdin` pipe, allowing input into the process
2. The `stdout` pipe, for reading output from the process
3. The `stderr` pipe, for reading error data from the process

Under the hood, both the `call` and `check_output` functions use a `Popen` object. To simulate `check_output` you can pass in a `PIPE` to the `stdout` parameter:

In [8]:
proc = subprocess.Popen(['ipconfig'], stdout=subprocess.PIPE)

In [9]:
out, err = proc.communicate()

print(out)


Windows IP Configuration


Ethernet adapter Ethernet:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::c42a:e633:dbe5:1438%2
   IPv4 Address. . . . . . . . . . . : 10.0.2.15
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 10.0.2.2

Tunnel adapter Teredo Tunneling Pseudo-Interface:

   Connection-specific DNS Suffix  . : 
   IPv6 Address. . . . . . . . . . . : 2001:0:9d38:6abd:1099:1929:f5ff:fdf0
   Link-local IPv6 Address . . . . . : fe80::1099:1929:f5ff:fdf0%4
   Default Gateway . . . . . . . . . : ::



Note that the `communicate` method will read all output from the process and wait for it to terminate. Thus you cannot `communicate` with a process a second time. Also note that without passing in a `PIPE` you cannot communicate with a process on that channel. So in the above example, even if there was an error written to the `stderr` stream, you would not see it.

Instead if you want to progressively communicate with a process, write to the input and output pipes as if you were writing to and from a file object. **WARNING** Reading from a PIPE is a blocking operation. If the process you are communicating with   does not immediately respond on the stdout, then reading may block your Python process.

In [10]:
proc = subprocess.Popen(['ipconfig'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [11]:
print(proc.stdout.read())


Windows IP Configuration


Ethernet adapter Ethernet:

   Connection-specific DNS Suffix  . : 
   Link-local IPv6 Address . . . . . : fe80::c42a:e633:dbe5:1438%2
   IPv4 Address. . . . . . . . . . . : 10.0.2.15
   Subnet Mask . . . . . . . . . . . : 255.255.255.0
   Default Gateway . . . . . . . . . : 10.0.2.2

Tunnel adapter Teredo Tunneling Pseudo-Interface:

   Connection-specific DNS Suffix  . : 
   IPv6 Address. . . . . . . . . . . : 2001:0:9d38:6abd:1099:1929:f5ff:fdf0
   Link-local IPv6 Address . . . . . : fe80::1099:1929:f5ff:fdf0%4
   Default Gateway . . . . . . . . . : ::



In [12]:
proc.poll()

0

### Exercise: Call a subprocess with `Popen`

Repeat the previous exercise, replacing `check_output` with `Popen` to check the active python processes in the system.

In [13]:
#%load 4000_solution_02.py

## Chaining processes together with pipes

Since the inputs and outputs of a subprocess are pipes, you can set the input pipe from another process, and use the output pipe into yet more processes. In this way you can chain together commands between disparate processes.

Consider this example: you have CSV data being read by your Python script (perhaps from a compresses dataset). You would rather this data in new-line separated JSON format. You could write your own Python script to process this data, but suppose you have the [jq](https://stedolan.github.io/jq/) and [csvkit](https://csvkit.readthedocs.io/en/1.0.3/) tools on your computer, which are optimised to work with JSON and CSV respectively.

To solve this problem you create two processes, one calling the `csvjson` tool, and link it to another calling `jq` to process the data.

In [14]:
csvjson = subprocess.Popen(['csvjson'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [15]:
jq = subprocess.Popen(['jq', '-c', '.[]'], stdin=csvjson.stdout, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [16]:
csvjson.stdin.write('name,id\r\nHannah,1\r\nMarcie,2\r\n')

In this case, the `csvjson` tool does not produce data until reaching the end of the file. Until we close the pipe it has not been reached.

In [17]:
csvjson.stdin.close()
csvjson.poll()

0

In [19]:
output, errors = jq.communicate()
print(output)

{"name":"Hannah","id":1}
{"name":"Marcie","id":2}



### Exercise: Connecting two commands via pipes

Use the `type` program on Windows (or `cat` on Unix) to read the `hamlet.txt` file from the data folder. Pipe it to the `clip` command. Confirm this has worked by pasting the text from your clipboard.

In [27]:
#%load 4000_solution_03.py

## Monitoring a log file

The Unix command `tail -F` follows a file. As new lines are added to the file by another process, `tail` updates the display. This is particularly useful for monitoring log files:

In [ ]:
tail -F /var/adm/messages

This displays the last 10 lines and then blocks until more lines are added to the file, then displays additional lines too.

Here is how you can call `tail -F` from Python using the `subprocess` module:

In [ ]:
import time
import subprocess
import sys

f = subprocess.Popen(['tail', '-F', '/var/log/some_log_file.log'],\
        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

while True:
    line = f.stdout.readline()
    line_text = line.decode(errors='ignore')
    sys.stdout.write(line)      # or do_something_with(line)

You could of course replace the final line with any other processing you like on the new line of the log file.

Note that the above solution is blocking: each call to `f.stdout.readline()` waits until a line is added to the file. A non-blocking version using `select` system call on Linux is here: https://stackoverflow.com/a/12523371

## `sh` module

It is worth mentioning that the `sh` module has a simpler interface for interacting with operating system processes. Here is an example that follows a file using `tail`:

In [ ]:
from sh import tail
# runs forever
for line in tail("-f", "/var/log/some_log_file.log", _iter=True):
    print(line)